In [1]:
import os
import sys
import torch
import numpy as np
import argparse
import torchvision.transforms as T
from data_utils import *

from torch.utils.data import DataLoader, Dataset
import pickle
import PIL
os.chdir("..")
from src.utils import *

In [8]:
for subject in tqdm(range(1,9)):
    create_whole_region_imagery_unnormalized(subject, mask=True)
    create_whole_region_imagery_normalized(subject, mask=True)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [01:05<00:00,  8.25s/it]


In [ ]:
def zscore(x, mean=None, stddev=None, return_stats=False):
    new_x = x
    if mean is not None:
        m = mean
    else:
        m = torch.mean(new_x, axis=0, keepdims=True)
    if stddev is not None:
        s = stddev
    else:
        s = torch.std(new_x, axis=0, keepdims=True)
    if return_stats:
        return (x - m)/(s+1e-6), m, s
    else:
        x = torch.where(s==0, (new_x - m), (new_x - m)/s)
        return x

In [ ]:
# nsd_root = '/export/raid1/home/surly/raid1/kendrick-data/nsd/'
# stim_dir = nsd_root + 'nsddata_stimuli/stimuli/nsd/'
# beta_dir = nsd_root + 'nsddata_betas/ppdata/'
# mask_dir= nsd_root + 'nsddata/ppdata/'
# img_stim_file = stim_dir + "nsdimagery_stimuli.pkl3"
# beta_subj = beta_dir + "subj%02d/func1pt8mm/nsdimagerybetas_fithrf/betas_nsdimagery.nii.gz"%subj

def image_feature_fn(image):
    '''take uint8 image and return floating point (0,1), either color or bw'''
    return image.astype(np.float32) / 255

## LOAD THE STIM IMAGES AND SEQUENCE ALIGNMENT DESCRIPTORS
stim_dir = "data/nsddata_stimuli/stimuli/nsd/"
img_stim_file = stim_dir + "nsdimagery_stimuli.pkl3"
ex_file = open(img_stim_file, 'rb')
imagery_dict = pickle.load(ex_file)
print(imagery_dict.keys())
ex_file.close()
exps = imagery_dict['exps']
cues = imagery_dict['cues']
image_map  = imagery_dict['image_map']
image_data = imagery_dict['image_data']
# for i, im in enumerate(image_data):
#     image = Image.fromarray(im.transpose(1,2,0))
#     image.save("data/nsddata_stimuli/stimuli/imagery_images/{}.png".format(i))

dict_keys(['meta_dm', 'gdm', 'gcm', 'image_data', 'image_map', 'exps', 'cues', 'modes'])


In [ ]:
def zscore(x, mean=None, stddev=None, return_stats=False):
    if mean is not None:
        m = mean
    else:
        m = np.mean(x, axis=0, keepdims=True)
    if stddev is not None:
        s = stddev
    else:
        s = np.std(x, axis=0, keepdims=True)
    if return_stats:
        return (x - m)/(s+1e-6), m, s
    else:
        return (x - m)/(s+1e-6)


## EXAMPLE CONDITION AVERAGED RESPONSES
def condition_average(data, cond):
    idx, idx_count = np.unique(cond, return_counts=True)
    idx_list = [cond==i for i in np.sort(idx)]
    avg_data = np.zeros(shape=(len(idx),)+data.shape[1:], dtype=np.float32)
    for i,m in enumerate(idx_list):
        avg_data[i] = np.mean(data[m], axis=0)
    return avg_data

meta_cond_idx = {
    'visA': np.arange(len(exps))[exps=='visA'],
    'visB': np.arange(len(exps))[exps=='visB'],
    'imgA_1': np.arange(len(exps))[exps=='imgA_1'],
    'imgA_2': np.arange(len(exps))[exps=='imgA_2'],
    'imgB_1': np.arange(len(exps))[exps=='imgB_1'],
    'imgB_2': np.arange(len(exps))[exps=='imgB_2']
}

cond_idx = {
    'visA': np.arange(len(exps))[exps=='visA'],
    'visB': np.arange(len(exps))[exps=='visB'],
    'imgA': np.arange(len(exps))[np.logical_or(exps=='imgA_1', exps=='imgA_2')],
    'imgB': np.arange(len(exps))[np.logical_or(exps=='imgB_1', exps=='imgB_2')]
}


In [ ]:
## LOAD THE DATA WITHOUT GLOBAL ZSCORING (EVEN IF IT IS ZSCORED WE ARE GONNA RE-ZSCORE LATER)

# voxel_data_raw = {}
# subjects = [1,2,3,4,5,6,7,8]
# for k,s in enumerate(subjects):
#     # print ('--------  subject %d  -------' % s)
#     create_whole_region_imagery_unnormalized(s)
#     create_whole_region_imagery_normalized(s)
    # voxel_data_raw[s] = torch.load("data/preprocessed_data/subject{}/nsd_imagery_unnormalized.pt".format(s))
    # print (voxel_data_raw[s].shape)


# ## NORMALIZATION OF THE DATA FOR EACH INDIVIDUAL TRIAL
# voxel_data_n = {}
# for s in voxel_data_raw.keys():
#     voxel_data_n[s] = np.copy(voxel_data_raw[s])
#     for c,idx in meta_cond_idx.items():
#         voxel_data_n[s][idx] = zscore(voxel_data_raw[s][idx])
#     print(voxel_data_n[s])


# cond_im_idx = {n: [image_map[c] for c in cues[idx]] for n,idx in cond_idx.items()}
# ## EXAMPLE USE
# for c, idx, im_idx in zip_dict(cond_idx, cond_im_idx): # loop conditions
#     data_single = voxel_data_n[s][idx]
#     data = condition_average(data_single, im_idx)
#     print(data.shape)

In [3]:
x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="imagery", stimtype="all", average=False, nest=True)
x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="vision", stimtype="all", average=False, nest=True)
# x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="imagery", stimtype="all", average=False, nest=True)
# x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="vision", stimtype="all", average=False, nest=True)


torch.Size([720, 15724])
torch.Size([12, 16, 15724]) torch.Size([12, 1024])
torch.Size([720, 15724])
torch.Size([12, 8, 15724]) torch.Size([12, 1024])


In [2]:
vector = "c"
subject = 1
mode = "imagery"
# stimtype="all"
stimtype="concepts"
average=False
nest=True

img_stim_file = "data/nsddata_stimuli/stimuli/nsd/nsdimagery_stimuli.pkl3"
ex_file = open(img_stim_file, 'rb')
imagery_dict = pickle.load(ex_file)
ex_file.close()
exps = imagery_dict['exps']
cues = imagery_dict['cues']
image_map  = imagery_dict['image_map']
image_data = imagery_dict['image_data']
cond_idx = {
'visionsimple': np.arange(len(exps))[exps=='visA'],
'visioncomplex': np.arange(len(exps))[exps=='visB'],
'visionconcepts': np.arange(len(exps))[exps=='visC'],
'visionall': np.arange(len(exps))[np.logical_or(exps=='visA', exps=='visB')],
'imagerysimple': np.arange(len(exps))[np.logical_or(exps=='imgA_1', exps=='imgA_2')],
'imagerycomplex': np.arange(len(exps))[np.logical_or(exps=='imgB_1', exps=='imgB_2')],
'imageryconcepts': np.arange(len(exps))[np.logical_or(exps=='imgC_1', exps=='imgC_2')],
'imageryall': np.arange(len(exps))[np.logical_or(np.logical_or(exps=='imgA_1', exps=='imgA_2'), np.logical_or(exps=='imgB_1', exps=='imgB_2'))]
}
x = torch.load("data/preprocessed_data/subject{}/nsd_imagery.pt".format(subject)).requires_grad_(False).to("cpu")
if stimtype != "concepts":
    cond_im_idx = {n: [image_map[c] for c in cues[idx]] for n,idx in cond_idx.items()}
    y = torch.load("data/preprocessed_data/{}_12.pt".format(vector)).requires_grad_(False).to("cpu")
else:
    cond_im_idx = {n: list(cues[idx]) for n,idx in cond_idx.items()}
    concept_map = {"S" : "stripes", "Z" :"zebra", "M" : "mammal", "Y" : "yellow", "N" : "banana", "F" : "fruit"}
    y = ["stripes", "zebra", "mammal", "yellow", "banana", "fruit"]

# Prune down to specific experimental mode/stimuli type
x = x[cond_idx[mode+stimtype]]
averaged_x, sample_count = condition_average(x, cond_im_idx[mode+stimtype])
# Letter cues get sorted out of order, so we need to fix the order
trial_count = int(x.shape[0]/sample_count)
# Average across trials
if average:
    x =  averaged_x
    if stimtype == "concepts":
        order = [3, 5, 1, 4, 2, 0]
        x = x[order]
    x = x.reshape((x.shape[0], 1, x.shape[1]))
    
elif nest:
    x_new = torch.zeros((sample_count, trial_count, x.shape[1]))
    for i in range(sample_count):
        x_new[i] = x[i*trial_count: i*trial_count + trial_count]
    x = x_new


if stimtype == "simple":
    y = y[:sample_count]
elif stimtype == "complex":
    y = y[sample_count:]
print(x.shape, y)

torch.Size([6, 16, 15724]) ['stripes', 'zebra', 'mammal', 'yellow', 'banana', 'fruit']
